In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
cluster.scale(140) 


In [3]:
from dask.distributed import Client
client = Client(cluster)


In [4]:
cluster

In [5]:


client



Client Scheduler: tcp://172.30.100.3:33845 Dashboard: http://172.30.100.3:41066/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import pandas as pd

today=date.today()

In [7]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [8]:
boxes_LS=pd.read_csv('../make_boxes/boxes_LS_1x1_eNATL60.csv',sep = '\t',index_col=0)
imin_LS=boxes_LS['imin']
imax_LS=boxes_LS['imax']
jmin_LS=boxes_LS['jmin']
jmax_LS=boxes_LS['jmax']
box_name=boxes_LS.index

In [9]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [10]:
def save_profile_dx(data_dx,data,var,imin,imax,jmin,jmax,profile_name,box):
    if not os.path.exists(profile_name):
            profile=data_dx[:,:,jmin:jmax,imin:imax].mean(dim={'x','y','time_counter'})
            dataset=profile.to_dataset(name='dx'+var)
            dataset['dx'+var].attrs=data.attrs
            dataset['dx'+var].attrs['standard_name']='dx gradient of '+data.attrs['standard_name']
            dataset['dx'+var].attrs['long_name']='dx_'+data.attrs['long_name']
            dataset['dx'+var].attrs['units']='degC/m'        
            dataset.attrs['global_attribute']= 'x-gradient of '+var+' profile averaged over 24h and in '+box+' computed on occigen with dask-jobqueue '+str(today)
            dataset.to_netcdf(path=profile_name,mode='w')


In [11]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV','vovecrtz' : 'gridW'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthv','vovecrtz':'depthw'}
filee1 = {'votemper' : 'e1t','vosaline' : 'e1t','vozocrtx' : 'e1u', 'vomecrty' : 'e1v','vovecrtz':'e1f'}


def compute_mean_dx_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,filedeps[var]:1})
    data=ds[str(var)]
    e1=dsgrid[str(filee1[var])][0]
    data_dx=dx_var(data,e1)
    for i in range(len(box_name)):
        box = box_name[i]
        print(box)  
        profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60'+box+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_dx'+var+'-profile.nc'
        %time save_profile_dx(data_dx,data,var,imin_LS[i],imax_LS[i],jmin_LS[i],jmax_LS[i],profile_name,box)

In [ ]:
%time compute_mean_dx_profile('vosaline','20090701')

Box_0
CPU times: user 1.25 ms, sys: 51 µs, total: 1.3 ms
Wall time: 1.2 ms
Box_1
CPU times: user 1.05 ms, sys: 0 ns, total: 1.05 ms
Wall time: 942 µs
Box_2
CPU times: user 475 µs, sys: 0 ns, total: 475 µs
Wall time: 375 µs
Box_3
CPU times: user 232 µs, sys: 0 ns, total: 232 µs
Wall time: 245 µs
Box_4
CPU times: user 0 ns, sys: 295 µs, total: 295 µs
Wall time: 357 µs
Box_5
CPU times: user 203 µs, sys: 74 µs, total: 277 µs
Wall time: 315 µs
Box_6
CPU times: user 301 µs, sys: 65 µs, total: 366 µs
Wall time: 271 µs
Box_7
CPU times: user 1.19 ms, sys: 0 ns, total: 1.19 ms
Wall time: 1.09 ms
Box_8
CPU times: user 714 µs, sys: 0 ns, total: 714 µs
Wall time: 600 µs
Box_9
CPU times: user 560 µs, sys: 0 ns, total: 560 µs
Wall time: 457 µs
Box_10
CPU times: user 216 µs, sys: 46 µs, total: 262 µs
Wall time: 532 µs
Box_11
CPU times: user 295 µs, sys: 0 ns, total: 295 µs
Wall time: 326 µs
Box_12
CPU times: user 286 µs, sys: 0 ns, total: 286 µs
Wall time: 255 µs
Box_13
CPU times: user 284 µs, sys: 0 

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.3:58620 remote=tcp://172.30.100.3:33845>


CPU times: user 55.8 s, sys: 3.4 s, total: 59.2 s
Wall time: 1min
Box_22
CPU times: user 1min 13s, sys: 5.17 s, total: 1min 18s
Wall time: 1min 18s
Box_23
CPU times: user 1min 3s, sys: 3.2 s, total: 1min 6s
Wall time: 1min 7s
Box_24


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


CPU times: user 1min, sys: 3.87 s, total: 1min 4s
Wall time: 1min 4s
Box_25


...

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


CPU times: user 1min 32s, sys: 4.2 s, total: 1min 37s
Wall time: 1min 37s
Box_40


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


CPU times: user 53.2 s, sys: 1.96 s, total: 55.2 s
Wall time: 56.4 s
Box_41


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


In [ ]:
%time compute_mean_dx_profile('votemper','20090701')

In [ ]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV','vovecrtz' : 'gridW'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthv','vovecrtz':'depthw'}
filee2 = {'votemper' : 'e2t','vosaline' : 'e2t','vozocrtx' : 'e2u', 'vomecrty' : 'e2v','vovecrtz':'e2f'}


def compute_mean_dy_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,filedeps[var]:1})
    data=ds[str(var)]
    e2=dsgrid[str(filee2[var])][0]
    data_dy=dy_var(data,e2)
    for i in range(len(box_name)):
        box = box_name[i]
        print(box)  
        profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60'+box.name+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_dy'+var+'-profile.nc'
        if not os.path.exists(profile_name):
            profile=data_dy[:,:,jmin_LS[i]:jmax_LS[i],imin_LS[i]:imax_LS[i]].mean(dim={'x','y','time_counter'})
            dataset=profile.to_dataset(name='dy'+var)
            dataset['dy'+var].attrs=data.attrs
            dataset['dy'+var].attrs['standard_name']='dy gradient of '+data.attrs['standard_name']
            dataset['dy'+var].attrs['long_name']='dy_'+data.attrs['long_name']
            dataset['dy'+var].attrs['units']='degC/m'        
            dataset.attrs['global_attribute']= 'y-gradient of '+var+' profile averaged over 24h and in '+box+' computed on occigen with dask-jobqueue '+str(today)
            dataset.to_netcdf(path=profile_name,mode='w')


In [ ]:
%time compute_mean_dx_profile('vozocrtx','20090701')

In [ ]:
%time compute_mean_dx_profile('vomecrty','20090701')

In [ ]:
%time compute_mean_dx_profile('vovecrtz','20090701')

In [ ]:
%time compute_mean_dy_profile('votemper','20090701')
%time compute_mean_dy_profile('vosaline','20090701')
%time compute_mean_dy_profile('vozocrtx','20090701')
%time compute_mean_dy_profile('vomecrty','20090701')
%time compute_mean_dy_profile('vovecrtz','20090701')
